In [174]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [175]:
data_dir = 'data/train/'

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  color_mode='grayscale',
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(28, 28),
  batch_size=10)

valid_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  color_mode='grayscale',
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(28, 28),
  batch_size=10)

normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
normalized_valid_ds = valid_ds.map(lambda x, y: (normalization_layer(x), y))



Found 261 files belonging to 3 classes.
Using 209 files for training.
Found 261 files belonging to 3 classes.
Using 52 files for validation.


In [176]:
class_names = train_ds.class_names
print(class_names)

['0', '1', '10']


In [177]:
# Show the first few images
# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#   for i in range(2):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(class_names[labels[i]])
#     plt.axis("off")
#     plt.show()

In [178]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(len(class_names))
])

In [179]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_18 (Flatten)        (None, 784)               0         
                                                                 
 dense_36 (Dense)            (None, 128)               100480    
                                                                 
 dropout_18 (Dropout)        (None, 128)               0         
                                                                 
 dense_37 (Dense)            (None, 3)                 387       
                                                                 
Total params: 100,867
Trainable params: 100,867
Non-trainable params: 0
_________________________________________________________________


In [180]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [181]:
model.fit(
  normalized_train_ds,
  validation_data=normalized_valid_ds,
  epochs=5
)

Epoch 1/5
21/21 [==============================] - 0s 7ms/step - loss: 0.3889 - accuracy: 0.8565 - val_loss: 0.1763 - val_accuracy: 0.9423
Epoch 2/5
21/21 [==============================] - 0s 4ms/step - loss: 0.1107 - accuracy: 0.9569 - val_loss: 0.1409 - val_accuracy: 0.9615
Epoch 3/5
21/21 [==============================] - 0s 4ms/step - loss: 0.0420 - accuracy: 0.9952 - val_loss: 0.1148 - val_accuracy: 0.9615
Epoch 4/5
21/21 [==============================] - 0s 4ms/step - loss: 0.0251 - accuracy: 0.9952 - val_loss: 0.1165 - val_accuracy: 0.9615
Epoch 5/5
21/21 [==============================] - 0s 4ms/step - loss: 0.0156 - accuracy: 1.0000 - val_loss: 0.1182 - val_accuracy: 0.9615


In [182]:
import tf2onnx
spec = (tf.TensorSpec((None, 28, 28, 1), tf.float32, name="input"),)
output_path = "onnx/test010.onnx"

model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13, output_path=output_path)